# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
uk_food_db = mongo['uk_food']


In [4]:
# review the collections in our database
print(uk_food_db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
uk_food_establishments = uk_food_db['establishments']

In [6]:
uk_food_establishments

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
query = {"scores.Hygiene": 20}
print("Number of documents in result:", uk_food_establishments.count_documents(query))
print("First result:")
results = uk_food_establishments.find(query)
pprint(results[1])

Number of documents in result: 0
First result:


IndexError: no such item for Cursor instance

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': {'$eq': 20}}
hygiene20_results = uk_food_establishments.find(query)

# Use count_documents to display the number of documents in the result
count_results = uk_food_establishments.count_documents(query)
print(f'number of establishments with a hygiene score of 20: {count_results}')

# Display the first document in the results using pprint
print()
print("e.g of establishments with a hygiene score of 20:")
print()
pprint(hygiene20_results[0])

number of establishments with a hygiene score of 20: 0

e.g of establishments with a hygiene score of 20:



IndexError: no such item for Cursor instance

In [ ]:
# Convert the result to a Pandas DataFrame
hygiene20_results_df =pd.DataFrame(hygiene20_results)

# Display the number of rows in the DataFrame
print(f'Number of rows in hygiene20_results_df: {len(hygiene20_results_df)}')

# Display the first 10 rows of the DataFrame
hygiene20_results_df.head(10)


Number of rows in hygiene20_results_df: 0


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [ ]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
London_query = {'LocalAuthorityName': {'$regex': "London"},
         'RatingValue': {'$gte': '4'}}

# Use count_documents to display the number of documents in the result
count_results = uk_food_establishments.count_documents(London_query)
print(f'establishments with London as the Local Authority and has a RatingValue greater than or equal to 4: {count_results}')
print()

# Display the first document in the results using pprint
London_results = uk_food_establishments.find(London_query)
pprint(London_results[0])

establishments with London as the Local Authority and has a RatingValue greater than or equal to 4: 0



IndexError: no such item for Cursor instance

In [ ]:
# Convert the result to a Pandas DataFrame
London_results_df =pd.DataFrame(London_results)

# Display the number of rows in the DataFrame
print(f'Number of rows in London_results_df: {len(London_results_df)}')

# Display the first 10 rows of the DataFrame
London_results_df.head(10)

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

within_query = {'RatingValue': {'$eq': '5'},
         'geocode.longitude': {'$gt':(longitude-degree_search),'$lt':(longitude+degree_search)},
         'geocode.latitude': {'$gt':(latitude-degree_search), '$lt':(latitude+degree_search)}
         }
sort = [('scores.Hygiene', -1)]
limit = 5


# Print the results
within_result = (list(uk_food_establishments.find(within_query).sort(sort).limit(limit)))
pprint(within_result)

In [ ]:
# Convert result to Pandas DataFrame
within_result_df = pd.DataFrame(within_result)

### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result
group_query = {'$group': {'_id':'$LocalAuthorityName', 'count':{'$sum': 1 }}}
match_query = {'$match': {'scores.Hygiene': {'$eq':0}}}
sort_values = {'$sort': {'count':-1}}

pipeline = [match_query, group_query, sort_values]
results = list(uk_food_establishments.aggregate(pipeline))
print("Rows in DataFrame: ", len(result_df))
print()

# Print the first 10 results
pprint(results[0:10])


In [ ]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print("Rows in DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)